In [1]:
!pip install --upgrade pip

In [2]:
!pip install torch
!pip install transformers
!pip install emoji

In [3]:
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast, AutoTokenizer

In [4]:
df = pd.read_csv('data_pt_brazil_normalized_emoji_merged.csv')
df = df[['created_at', 'text', 'sentiment']]
df.head()

,created_at,text,sentiment
0,2021-07-04 22:17:21,RT @emirsader: Após ficar em silêncio na CPI d...,Positivo
1,2019-11-30 14:35:33,RT @VittorGuidoni: para de lamber fone de ouvi...,Negativo
2,2022-01-11 12:12:28,@exposed_exposer @CarlaZambelli38 @andrizek De...,Negativo
3,2019-11-19 14:52:06,RT @canaltech: Por que o iPhone é mais seguro ...,Positivo
4,2021-12-28 23:09:10,RT @mariareinhardtt: covid tá a estragar compl...,Negativo


In [5]:
emoji_bank = {
  ",:(": "😓",
  ",:)": "😅",
  ",:-(": "😓",
  ",:-)": "😅",
  "0:)": "😇",
  "0:-)": "😇",
  "8-)": "😎",
  ":$": "😒",
  ":'(": "😢",
  ":')": "😂",
  ":'-(": "😢",
  ":'-)": "😂",
  ":'-D": "😂",
  ":'D": "😂",
  ":(": "😦",
  ":)": "😃",
  ":*": "😗",
  ":,'(": "😭",
  ":,'-(": "😭",
  ":,(": "😢",
  ":,)": "😂",
  ":,-(": "😢",
  ":,-)": "😂",
  ":,-D": "😂",
  ":,D": "😂",
  ":-$": "😒",
  ":-(": "😦",
  ":-)": "😃",
  ":-*": "😗",
  ":-/": "😕",
  ":-@": "😡",
  ":-D": "😄",
  ":-o": "😮",
  ":-O": "😮",
  ":-P": "😛",
  ":-S": "😒",
  ":-Z": "😒",
  ":-|": "😐",
  ":/": "😕",
  ":@": "😡",
  ":D": "😄",
  ":o": "😮",
  ":O": "😮",
  ":P": "😛",
  ":s": "😒",
  ":z": "😒",
  ":|": "😐",
  ";(": "😭",
  ";)": "😉",
  ";-(": "😭",
  ";-)": "😉",
  "]:)": "😈",
  "]:-)": "😈",
  "B-)": "😎",
  "o:)": "😇",
  "O:)": "😇",
  "O:-)": "😇",
  "o:-)": "😇",
  "X-)": "😆",
  "x-)": "😆"
}

In [6]:
stopwords_list = ['rt']

In [7]:
import re, string, unicodedata
import nltk
import emoji

def extract_emojis(sample):
    return ' '.join(c for c in sample if c in emoji.EMOJI_DATA).split()

def remove_user_from_text(words):
    return " ".join(filter(lambda x:x[0]!='@', words.split()))

def remove_double_space(sample):
    return " ".join(sample.split())

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    return list(map(lambda x: x.lower(), words))

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def clean_text(sample):
    emoji_pat = '[\U0001F300-\U0001F64F\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]'
    shrink_whitespace_reg = re.compile(r'\s{2,}')
    reg = re.compile(r'({})|[^a-zA-Z]'.format(emoji_pat)) # line a
    result = reg.sub(lambda x: ' {} '.format(x.group(1)) if x.group(1) else ' ', sample)
    return shrink_whitespace_reg.sub(' ', result)

def replace_asci_emoji(sample):
    address = sample
    for k,v in emoji_bank.items():
        address = address.replace(k, v)
    return address

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords_list:
            new_words.append(word)
    return new_words

def normalize(words):
    words = remove_user_from_text(words)
    words = to_lowercase(words.split())
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_asci_emoji(sample)
#     sample = clean_text(sample)
    sample = remove_double_space(sample)
    return normalize(sample)

In [8]:
df['normalized'] = df.apply(lambda x: " ".join(preprocess(x['text'])), axis=1)
df.head()

,created_at,text,sentiment,normalized
0,2021-07-04 22:17:21,RT @emirsader: Após ficar em silêncio na CPI d...,Positivo,após ficar em silêncio na cpi da covid carlos ...
1,2019-11-30 14:35:33,RT @VittorGuidoni: para de lamber fone de ouvi...,Negativo,para de lamber fone de ouvido lista de doenças...
2,2022-01-11 12:12:28,@exposed_exposer @CarlaZambelli38 @andrizek De...,Negativo,definição de vacina é um tipo de substância ví...
3,2019-11-19 14:52:06,RT @canaltech: Por que o iPhone é mais seguro ...,Positivo,por que o iphone é mais seguro contra vírus
4,2021-12-28 23:09:10,RT @mariareinhardtt: covid tá a estragar compl...,Negativo,covid tá a estragar completamente tudo


In [9]:
df = df[['created_at', 'sentiment', 'normalized']]
df.head()

,created_at,sentiment,normalized
0,2021-07-04 22:17:21,Positivo,após ficar em silêncio na cpi da covid carlos ...
1,2019-11-30 14:35:33,Negativo,para de lamber fone de ouvido lista de doenças...
2,2022-01-11 12:12:28,Negativo,definição de vacina é um tipo de substância ví...
3,2019-11-19 14:52:06,Positivo,por que o iphone é mais seguro contra vírus
4,2021-12-28 23:09:10,Negativo,covid tá a estragar completamente tudo


In [10]:
df['sentiment'] = df['sentiment'].replace(['Positivo','Negativo','Neutro'],[2,0,1])

In [11]:
df['sentiment'].value_counts(normalize = True)

0    0.467800
1    0.294875
2    0.237325
Name: sentiment, dtype: float64

In [12]:
df = df.sample(n=20000, random_state=42).reset_index(drop=True)

In [13]:
# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['normalized'], df['sentiment'], 
                                                                    random_state=42, 
                                                                    test_size=0.3, 
                                                                    stratify=df['sentiment'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=42, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [14]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
bert = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
max_seq_len = 140

In [16]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2383: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [17]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [18]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [19]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [20]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
        super(BERT_Arch, self).__init__()

        self.bert = bert 
      
      # dropout layer
        self.dropout = nn.Dropout(0.1)
      
      # relu activation function
        self.relu =  nn.ReLU()

      # dense layer 1
        self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,3)

      #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask,return_dict=False)
      
        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

      # output layer
        x = self.fc2(x)
      
      # apply softmax activation
        x = self.softmax(x)

        return x

In [21]:
torch.cuda.is_available = lambda : False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device("cuda")

In [22]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [23]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

/usr/local/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [24]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[0.7019655  1.13793384 1.43545576]


/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1 2], y=9386     0
3665     0
1018     0
15040    1
5089     1
        ..
12482    1
1110     2
7877     0
15786    0
15623    1
Name: sentiment, Length: 14000, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [25]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

In [26]:
# function to train the model
def train():
  
    model.train()

    total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
    total_preds=[]
  
  # iterate over batches
    for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
        batch = [r.to(device) for r in batch]
 
        sent_id, mask, labels = batch

    # clear previously calculated gradients 
        model.zero_grad()        

    # get model predictions for the current batch
        preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

    # add on to the total loss
        total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
        loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
        optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

    # append the model predictions
        total_preds.append(preds)

  # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
    return avg_loss, total_preds

In [27]:
# function for evaluating the model
def evaluate():
  
    print("\nEvaluating...")
  
  # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
    total_preds = []

  # iterate over batches
    for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            
      # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

    # deactivate autograd
        with torch.no_grad():
      
      # model predictions
            preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

  # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [28]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of    438.
  Batch   100  of    438.
  Batch   150  of    438.
  Batch   200  of    438.
  Batch   250  of    438.
  Batch   300  of    438.
  Batch   350  of    438.
  Batch   400  of    438.

Evaluating...
  Batch    50  of     94.

Training Loss: 1.044
Validation Loss: 1.035

 Epoch 2 / 10
  Batch    50  of    438.
  Batch   100  of    438.
  Batch   150  of    438.
  Batch   200  of    438.
  Batch   250  of    438.
  Batch   300  of    438.
  Batch   350  of    438.
  Batch   400  of    438.

Evaluating...
  Batch    50  of     94.

Training Loss: 1.012
Validation Loss: 0.990

 Epoch 3 / 10
  Batch    50  of    438.
  Batch   100  of    438.
  Batch   150  of    438.
  Batch   200  of    438.
  Batch   250  of    438.
  Batch   300  of    438.
  Batch   350  of    438.
  Batch   400  of    438.

Evaluating...
  Batch    50  of     94.

Training Loss: 0.996
Validation Loss: 0.995

 Epoch 4 / 10
  Batch    50  of    438.
  Batch   100  of    438.
  Batc

In [30]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [31]:
len(test_seq)

3000

In [32]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq[0:3000].to(device), test_mask[0:3000].to(device))
    preds = preds.detach().cpu().numpy()

In [34]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y[0:3000], preds[0:3000]))

              precision    recall  f1-score   support

           0       0.64      0.68      0.66      1425
           1       0.57      0.49      0.53       879
           2       0.41      0.44      0.43       696

    accuracy                           0.57      3000
   macro avg       0.54      0.54      0.54      3000
weighted avg       0.57      0.57      0.57      3000



In [48]:
preds = np.argmax(preds, axis = 1)

In [49]:
preds

array([2, 0, 0, ..., 0, 1, 2])